In [0]:
ApplicationClientID = dbutils.secrets.get(scope = "key-vault-scope", key = "ApplicationClientID")
SecretKey = dbutils.secrets.get(scope = "key-vault-scope", key = "SecretKey")
DirectoryTenantID = dbutils.secrets.get(scope = "key-vault-scope", key = "DirectoryTenantID")

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": ApplicationClientID,
       "fs.azure.account.oauth2.client.secret": SecretKey,
       "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{DirectoryTenantID}/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:   
       dbutils.fs.unmount('/mnt/tokyoolimpic')
       dbutils.fs.mount(
       source = "abfss://tokyoolimpic@tokyoolympicdatadb.dfs.core.windows.net",
       mount_point = "/mnt/tokyoolimpic",
       extra_configs = configs)
except Exception as e:
       if "Directory already mounted" not in e.message:
              raise e

In [0]:
dbutils.fs.ls("/mnt/tokyoolimpic/raw/")  

In [0]:
spark